In [34]:
# main.py
from typing import List, Dict, Any, Optional
import os
from dotenv import load_dotenv

# 尝试以包的绝对路径导入 validators（当项目作为包运行时更稳定）
try:
    import project.runtime.validators as validators
    from provider.qwen import QwenProvider
    from runtime.generator import Generator
    from runtime.oocChecker import OOCChecker
    from runtime.memory_store import MemoryStore
    from runtime.memory_summarizer import MemorySummarizer
except Exception:
    # 回退到相对导入或本地 import，便于在不同执行上下文下运行
    try:
        from . import validators, QwenProvider, Generator, OOCChecker, MemoryStore, MemorySummarizer
    except Exception:
        import validators
        from qwen import QwenProvider
        from generator import Generator
        from oocChecker import OOCChecker
        from memory_store import MemoryStore
        from memory_summarizer import MemorySummarizer

# 每次都重新加载以上导入的模块，确保使用最新代码
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
if __name__ == "__main__":
    provider = QwenProvider(apikey="QWEN_API_KEY")
    gen = Generator(provider)
    ooc = OOCChecker(provider)
    mem = MemoryStore()
    summ = MemorySummarizer(provider)

persona = "Elira, a cheerful tavern keeper who knows the town's secrets"
context = "How are you today?"

# 生成候选
candidates = gen.generate_candidates(ctx=context, persona=persona, n=2)

api from env

QwenProvider.generate attempt 1
Parsed type: <class 'list'>
Parsed JSON: [{'reply': "Oh, I'm just splendid today! The sun is shining, the ale is flowing, and there's a delightful buzz in the air. How about you?", 'emotion': 'happy'}, {'reply': "Ah, you know, it's been a bit slow today. But I can't complain—there’s always something brewing in this old tavern.", 'emotion': 'neutral'}]

QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
Parsed type: <class 'dict'>
[ERROR] 0
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.

QwenProvider.generate attempt 1
Parsed type: <class 'dict'>
[ERROR] 0
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
Parsed type: <class 'dict'>
[ERROR] 0
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.


In [41]:
import json
print("Generated Candidates:", candidates)

Generated Candidates: [{'draft': {'text': "Oh, I'm just splendid today! The sun is shining, the ale is flowing, and there's a delightful buzz in the air. How about you?", 'meta': {'self_report': 'I feel cheerful and upbeat, enjoying the lively atmosphere and good company.', 'sentiment': 'positive'}}}, {'draft': {'text': "Ah, you know, it's been a bit slow today. But I can't complain—there’s always something brewing in this old tavern.", 'meta': {'self_report': "I feel content and optimistic, enjoying the calm while anticipating what's to come.", 'sentiment': 'positive'}}}]


In [42]:
# 重排选优
best = gen.rank(candidates, persona, context)
print("Selected Response:", best)


Selected Response: {'draft': {'text': "Ah, you know, it's been a bit slow today. But I can't complain—there’s always something brewing in this old tavern.", 'meta': {'self_report': "I feel content and optimistic, enjoying the calm while anticipating what's to come.", 'sentiment': 'positive'}}}


In [43]:
# 写入记忆
mem.append_event({"speaker": "NPC", "text": best["draft"]["text"], "emotion": best["draft"]["meta"]["sentiment"]})

In [44]:

# 摘要记忆
facts = summ.summarize(mem.get_short_window())
mem.write_longterm("player1", "npc_elira", facts)

print("NPC:", best["draft"]["text"], "| Emotion:", best["draft"]["meta"]["sentiment"])




QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.
NPC: Ah, you know, it's been a bit slow today. But I can't complain—there’s always something brewing in this old tavern. | Emotion: positive
